In [8]:
# 模块导入
import pandas as pd
from multiprocessing import Process, Queue,Lock
from concurrent.futures import ProcessPoolExecutor

from geopy.distance import geodesic
import time

In [9]:
# 读入txt 每一行的数据形如:1,30.624806,104.136604,1,2014/8/3 21:18:46
# 1:出租车ID 2:维度 3:经度 4:载客情况(1为有,0为无) 5:时间
# 读入txt文件，将每个出租车的数据分别存入一个csv文件中
file_path="all_txt_data\\20140803_train.txt"
column_names=['taxi_id','latitude','longitude','passenger','time']
# 读入txt文件1
df=pd.read_csv(file_path,sep=',',header=None,names=column_names)
"""
原始数据读取

df 字段说明:
    taxi_id:出租车ID
    latitude:维度
    longitude:经度
    passenger:载客情况(1为有,0为无)
    time:时间
"""

'\n原始数据读取\n\ndf 字段说明:\n    taxi_id:出租车ID\n    latitude:维度\n    longitude:经度\n    passenger:载客情况(1为有,0为无)\n    time:时间\n'

# 数据预处理
将GPS轨迹数据转为空驶、载客的一段段轨迹数据，包括起始点、终止点、起止时间、终止时间、载客状态和轨迹长度

存储在`trip_df`中

In [10]:
# 提取出taxi_id为的数据
# df_selected = df[df['taxi_id']<=2000].copy() # 选择原始数据中一部分数据测试用
df_selected = df

# 把time列形如:2014/8/3 06:00:53的数据转换成易于处理的形式
df_selected['time']=pd.to_datetime(df_selected['time'],format='%Y/%m/%d %H:%M:%S')
# 增加一列time_s,表示以秒为单位的时间
df_selected['time_s']=df_selected['time'].dt.hour*3600+df_selected['time'].dt.minute*60+df_selected['time'].dt.second
# 以taxi_id为第一关键字，以time为第二关键字，对数据进行排序
df_selected=df_selected.sort_values(by=['taxi_id','time_s'])
# 删除time列
df_selected=df_selected.drop(['time'],axis=1)
# passenger改为bool型
df_selected['passenger']=df_selected['passenger'].astype('bool')
# 把int64型的数据转换成int32型，减少内存占用
df_selected['taxi_id']=df_selected['taxi_id'].astype('int16') # taxi_id不超过15000，可以用int16
df_selected['time_s']=df_selected['time_s'].astype('int32')
# 把float64型的数据转换成float32型，减少内存占用
df_selected['longitude']=df_selected['longitude'].astype('float32')
df_selected['latitude']=df_selected['latitude'].astype('float32')
# 直接以dataframe的形式保存
df_selected.to_pickle('df_selected.pkl')
# 打印出每一列的数据类型以及各自占用的内存大小
print(df_selected.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53045407 entries, 1261 to 53044761
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   taxi_id    int16  
 1   latitude   float32
 2   longitude  float32
 3   passenger  bool   
 4   time_s     int32  
dtypes: bool(1), float32(2), int16(1), int32(1)
memory usage: 1.1 GB
None


这里开始保存了df_selected，去concur.py里跑（由于多进程的某些原因，需要在main函数里跑）

下面这个块只是为了分析测试，不用管

In [11]:
# 每辆车的GPS数据采取的时间间隔最长和最短值
time_interval_min = df_selected.groupby('taxi_id')['time_s'].diff().min()
time_interval_max = df_selected.groupby('taxi_id')['time_s'].diff().max()
print('time_interval_min:',time_interval_min)
print('time_interval_max:',time_interval_max)

# 车辆总数
taxi_num = df_selected['taxi_id'].unique().shape[0]
print('taxi_num:',taxi_num)

# 找出每辆车 时间间隔为time_interval_min的数据
df_test = df_selected.groupby('taxi_id').apply(lambda x:x[x['time_s'].diff()==time_interval_min])
print(df_test)
# 找出每辆车 时间间隔为time_interval_max的数据
df_test = df_selected.groupby('taxi_id').apply(lambda x:x[x['time_s'].diff()==time_interval_max])
print(df_test)

time_interval_min: 0.0
time_interval_max: 61335.0
taxi_num: 13605
                  taxi_id   latitude   longitude  passenger  time_s
taxi_id                                                            
12      38555          12  30.597746  103.958145      False   31653
        39934          12  30.659048  104.077415      False   50052
28      81475          28  30.632174  104.137566      False   50245
        81439          28  30.629587  104.136360      False   50276
76      268963         76  30.647436  104.133156       True   52179
...                   ...        ...         ...        ...     ...
13578   52935588    13578  30.605471  104.151474      False   27759
        52935589    13578  30.605471  104.151474      False   27769
        52935590    13578  30.605471  104.151474      False   27779
        52935591    13578  30.605471  104.151474      False   27789
13585   52960960    13585  30.652187  104.078171       True   65913

[3579 rows x 5 columns]
                  taxi_id

跑完concur.py，下面进行一些筛除

In [3]:
trip_df = pd.read_pickle('trip_df.pkl')

In [4]:
# 检查有无路径长为0的轨迹
print(len(trip_df[trip_df['distance'] == 0]))
# 删除这些路径长为0的轨迹
trip_df = trip_df[trip_df['distance'] != 0]
print(len(trip_df[trip_df['distance'] == 0]))

0
0


In [5]:
# 检查有无行程时间为0的轨迹
print(len(trip_df[trip_df['start_time'] == trip_df['end_time']]))
# 删除这些行程时间为0的轨迹

trip_df = trip_df[trip_df['start_time'] != trip_df['end_time']]
print(len(trip_df[trip_df['start_time'] == trip_df['end_time']]))

0
0


In [6]:
""" 筛选出早高峰、午高峰、晚高峰的数据
早高峰:7:30-8:30
午高峰:14:00-15:00
晚高峰:17:00-19:00
"""

trip_df_morning = trip_df[(trip_df['start_time'] >= 7*3600+30*60) & (trip_df['start_time'] <= 8*3600+30*60) & (trip_df['end_time'] >= 7*3600+30*60) & (trip_df['end_time'] <= 8*3600+30*60)]
trip_df_noon = trip_df[(trip_df['start_time'] >= 14*3600) & (trip_df['start_time'] <= 15*3600) & (trip_df['end_time'] >= 14*3600) & (trip_df['end_time'] <= 15*3600)]
trip_df_night = trip_df[(trip_df['start_time'] >= 17*3600) & (trip_df['start_time'] <= 19*3600) & (trip_df['end_time'] >= 17*3600) & (trip_df['end_time'] <= 19*3600)]

In [7]:
# 保存
trip_df.to_pickle('trip_df.pkl')
trip_df_morning.to_pickle('trip_df_morning.pkl')
trip_df_noon.to_pickle('trip_df_noon.pkl')
trip_df_night.to_pickle('trip_df_night.pkl')